In [45]:
import os, glob, re
import shutil
import numpy as np
import pandas as pd
from astropy.table import Table
from tqdm.auto import tqdm
import matplotlib.pyplot as plt

In [3]:
df = Table.read("../../EXBA_LCFs/data/hlsp_kbonus-apexba_kepler_kepler_source-list_kepler_v1.0_cat.fits").to_pandas()

In [4]:
df

,Gaia_ID,ra,ra_error,dec,dec_error,pmra,pmdec,parallax,parallax_error,phot_g_mean_mag,...,umag,gmag,rmag,imag,zmag,Jmag,Hmag,Kmag,kepmag,angDist
0,b'Gaia EDR3 2105384211065299328',282.026014,0.190857,44.253779,0.223919,-19.973063,-18.944105,0.589230,0.221065,19.495558,...,NaN,NaN,19.629999,18.360001,NaN,NaN,NaN,NaN,19.629999,0.456638
1,b'Gaia EDR3 2105384241127278208',282.006752,0.132529,44.253545,0.153231,-1.771941,-2.529980,0.150096,0.155027,19.065722,...,NaN,19.868999,19.221001,18.988001,NaN,NaN,NaN,NaN,19.254000,0.326691
2,b'Gaia EDR3 2105384245425035392',282.014153,0.058103,44.253572,0.065566,-1.188432,-1.331190,0.649663,0.066387,17.685090,...,NaN,18.594999,17.742001,17.417999,NaN,15.890,15.292,15.129,17.771999,0.285838
3,b'Gaia EDR3 2105384245425036032',282.011585,0.046780,44.256934,0.052581,-0.148561,2.350119,0.418869,0.052441,17.299631,...,NaN,17.952000,17.243999,17.068001,17.049,15.903,15.364,15.289,17.333000,0.065174
4,b'Gaia EDR3 2105384275487021696',282.017593,0.112678,44.257041,0.131294,-3.580035,-7.079613,0.074377,0.129395,18.775005,...,NaN,19.363001,18.906000,18.764999,NaN,NaN,NaN,NaN,18.947001,0.376572
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9322,b'Gaia EDR3 2086539951857563136',298.064847,0.352570,48.204598,0.475988,-1.418608,-6.318064,0.542188,0.458016,20.171392,...,NaN,21.274000,20.294001,19.855000,NaN,NaN,NaN,NaN,20.283001,0.491628
9323,b'Gaia EDR3 2129526011778662272',290.697294,0.135908,48.615814,0.133261,-3.101519,-4.253412,0.115126,0.141861,18.787304,...,NaN,19.183001,18.629999,18.422001,NaN,NaN,NaN,NaN,18.660999,0.220906
9324,b'Gaia EDR3 2143714973114706560',282.740530,0.021523,48.262489,0.022695,-3.188320,-5.866775,0.588440,0.023253,15.417315,...,NaN,15.645000,15.162000,15.031000,NaN,14.322,14.068,14.005,15.216000,0.136016
9325,b'Gaia EDR3 2076208704382822912',295.766628,0.121251,39.445539,0.120965,-5.705135,-8.953973,0.692738,0.141725,18.638466,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
ids = np.array([x.decode("utf8").split(" ")[-1] for x in df.Gaia_ID.values])

# Test how many digits for first directory level

In [40]:
ndigits = [4,5,6]

for nd in ndigits:
    dir_names = list(set([x[:nd] for x in ids]))
    print(nd, ":", len(dir_names))
    hm_in_dir = np.zeros(len(dir_names))
    for k, pf in enumerate(dir_names):
        hm_in_dir[k] = np.sum([id.startswith(pf) for id in ids])
    print(hm_in_dir.min(), hm_in_dir.max())
    # plt.figure(figsize=(4,9))
    # plt.hist(hm_in_dir)
    # plt.show()
    # break

4 : 36
45.0 665.0
5 : 106
3.0 466.0
6 : 151
1.0 263.0


In [35]:
quarters = np.arange(5, 18)
channels = np.delete(np.arange(1, 85, 1), [4,5,6,7])
quarters, channels

(array([ 5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17]),
 array([ 1,  2,  3,  4,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21,
        22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38,
        39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55,
        56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72,
        73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84]))

# Count files in Q/Ch archive

In [96]:
fcounter = 0
for q in tqdm(quarters):
    for ch in tqdm(channels, leave=False):
        in_dir = "../../EXBA_LCFs/data/q%02i/ch%02i" % (q, ch)
        files = glob.glob("%s/hlsp_kbonus-apexba_kepler_kepler_gaia-edr3-*-q%02i_kepler_v1.0_lc.fits*" % (in_dir, q))
        # if q == 17:
        fcounter += len(files)
fcounter

  0%|          | 0/13 [00:00<?, ?it/s]

  0%|          | 0/80 [00:00<?, ?it/s]

  0%|          | 0/80 [00:00<?, ?it/s]

  0%|          | 0/80 [00:00<?, ?it/s]

  0%|          | 0/80 [00:00<?, ?it/s]

...

  0%|          | 0/80 [00:00<?, ?it/s]

  0%|          | 0/80 [00:00<?, ?it/s]

  0%|          | 0/80 [00:00<?, ?it/s]

  0%|          | 0/80 [00:00<?, ?it/s]

90224

# Move sup files

In [88]:
quarters[-1:]

array([17])

In [101]:
for q in tqdm(quarters):
    for ch in tqdm(channels):
        in_dir = "../../EXBA_LCFs/data_old_arxive/q%02i/ch%02i" % (q, ch)
        in_file = glob.glob("%s/hlsp_kbonus-apexba_kepler_kepler_exba-mask-ch%02i-q%02i_kepler_v1.0_sup.fits" % (in_dir, ch, q))[0]
        
        out_dir = "../../EXBA_LCFs/data/ch%02i" % (ch)
        if not os.path.isdir(out_dir):
            os.makedirs(out_dir)
        out_file = "%s/%s" % (out_dir, os.path.basename(in_file))
        # print(in_file, " --> ", out_file)
        shutil.copy2(in_file, out_file)

  0%|          | 0/13 [00:00<?, ?it/s]

  0%|          | 0/80 [00:00<?, ?it/s]

  0%|          | 0/80 [00:00<?, ?it/s]

  0%|          | 0/80 [00:00<?, ?it/s]

  0%|          | 0/80 [00:00<?, ?it/s]

...

  0%|          | 0/80 [00:00<?, ?it/s]

  0%|          | 0/80 [00:00<?, ?it/s]

  0%|          | 0/80 [00:00<?, ?it/s]

  0%|          | 0/80 [00:00<?, ?it/s]

  0%|          | 0/80 [00:00<?, ?it/s]

# Move LCFs

In [99]:
for q in tqdm(quarters):
    for ch in tqdm(channels):
        in_dir = "../../EXBA_LCFs/data_old_arxive/q%02i/ch%02i" % (q, ch)
        files = glob.glob("%s/hlsp_kbonus-apexba_kepler_kepler_gaia-edr3-*-q%02i_kepler_v1.0_lc.fits" % (in_dir, q))
        for i, f in enumerate(files):
            gid = f.split("-")[-2]
            out_dir = "../../EXBA_LCFs/data/%s/%s" % (gid[:6], gid)
            if not os.path.isdir(out_dir):
                os.makedirs(out_dir)
            # print(f, " --> ", "%s/%s" % (out_dir, os.path.basename(f)))
            shutil.copy2(f, "%s/%s" % (out_dir, os.path.basename(f)))
        # break
    # break

  0%|          | 0/13 [00:00<?, ?it/s]

  0%|          | 0/80 [00:00<?, ?it/s]

  0%|          | 0/80 [00:00<?, ?it/s]

  0%|          | 0/80 [00:00<?, ?it/s]

  0%|          | 0/80 [00:00<?, ?it/s]

...

  0%|          | 0/80 [00:00<?, ?it/s]

  0%|          | 0/80 [00:00<?, ?it/s]

  0%|          | 0/80 [00:00<?, ?it/s]

  0%|          | 0/80 [00:00<?, ?it/s]

  0%|          | 0/80 [00:00<?, ?it/s]

# Count files in HLSP archive

In [105]:
def glob_re(pattern, strings):
    return filter(re.compile(pattern).match, strings)

In [118]:
fcounter = 0
first_level = glob.glob("../../EXBA_LCFs/data/2*")
for fl in tqdm(first_level):
    second_level = glob.glob("%s/*" % fl)

    for sl in second_level:
        files = glob.glob("%s/hlsp_kbonus-apexba_kepler_kepler_gaia-edr3-*-q*_kepler_v1.0_lc.fits" % (sl))
        # if q == 17:
        fcounter += len(files)
        
    # break
fcounter

  0%|          | 0/151 [00:00<?, ?it/s]

90224